# Cut Summary
In this file, I compile a list of the cuts used in places in R76Analysis.

In [1]:
#Imports
import math
import sys
import numpy as np              # Will be used for binning
import pandas as pd             # Will be used for data structuring
import matplotlib.pyplot as plt # Basic plotting library
import lmfit as lf              # Fitting
from matplotlib import cm
from matplotlib import colors
sys.path.append('../')
import R76Tools as R76 # Importing it like this fixes some weirdness with names
gaus_mod = lf.Model(R76.gaus)
gaus_params = gaus_mod.make_params()
gaus_params.add('A',min=0,value=300)
gaus_params.add('sigma',min=0,value=3,max=200)
gaus_params.add('mu',min=100,max=300,value=150)

#Constants
frittspath = "/data/chocula/fritts/data/k100proc/midasrq/byseries/"

In [ ]:
datapath=frittspath
ser=["07220827_1153"]
lab=["0V","-4V","-21V","-65V","-84V","-84V"]
_,z = R76.makechain_list(ser,path=datapath,filters=[R76.fittingfilters,"PTOFamps0","PTOFdelay"],aliases=R76.qualityaliases)

# Cuts

In [10]:
print(dir(z[0]))
crand=[] # True if the event was randomly-triggered.
cam=[] # True if the event looks appropriate for one from americium.
cphi1=[]
cbs=[]
cgoodwalk=[]
cofintl=[]
cofintt=[]
cofintvl=[]
cz=[]
ccoin=[]
for i,x in enumerate(z):
    crand.append(x['EventCategory']==1)
    cam.append(x['PFOFamps']/x['PTOFamps']>0.21)
    cphi1.append((x['phidel']>5)&(x['phidel']<20))
    cbs.append(x["BSel"]<1100)
    cgoodwalk.append((x["PCWKr20"]>0.25e-3)&(x["PCWKr20"]<0.5e-3)&(x["PDWKr20"]>0.25e-3)&(x["PDWKr20"]<0.5e-3)&\
        (x["PEWKr20"]>0.25e-3)&(x["PEWKr20"]<0.5e-3))
    cofintl.append((x["PTINTall_PTbscorr"]/x["PTOFamps"]>50) & (x["PTINTall_PTbscorr"]/x["PTOFamps"]<750))
    cofintt.append((x["PTINTall_PTbscorr"]/x["PTOFamps"]>250) & (x["PTINTall_PTbscorr"]/x["PTOFamps"]<550))
    x['PTdbs']=(x['PAbspost']-x['PAbs']) + (x['PBbspost']-x['PBbs']) + (x['PCbspost']-x['PCbs']) + (x['PDbspost']-x['PDbs'])\
    + (x['PEbspost']-x['PEbs']) + (x['PFbspost']-x['PFbs'])
    x['PTINTall_PTdbscorr']=x['PTINTall']-(x['PTdbs']<0)*0.5*(x['PTdbs']*4096./x['PFnorm'])
    cofintvl.append((x['PTINTall_PTdbscorr']/x['PTOFamps']>-700)&(x['PTINTall_PTdbscorr']/x['PTOFamps']<1500))
    x["pt_keV_bscorr"] = x["pt_keV"]/(1-0.0461538*x["BSel"]/165)
    x["pt_keVee"] = 14.0821*x["pt_keV_bscorr"]/(333.899 - x["pt_keV_bscorr"])
    cz.append(~crand[i]&cbs[i]&(x["pt_keVee"]>0)&(x["pt_keVee"]<200))
    ccoin.append((x["PTOFdelay"]/0.8e-6 > 8) & (x["PTOFdelay"]/0.8e-6 < 40))

['BSel', 'EventCategory', 'PAOFamps', 'PAWKf20', 'PAWKf40', 'PAWKr20', 'PAWKr40', 'PAbs', 'PAbspost', 'PBOFamps', 'PBWKf20', 'PBWKf40', 'PBWKr20', 'PBWKr40', 'PBbs', 'PBbspost', 'PCOFamps', 'PCWKf20', 'PCWKf40', 'PCWKr20', 'PCWKr40', 'PCbs', 'PCbspost', 'PDOFamps', 'PDWKf20', 'PDWKf40', 'PDWKr20', 'PDWKr40', 'PDbs', 'PDbspost', 'PEOFamps', 'PEWKf20', 'PEWKf40', 'PEWKr20', 'PEWKr40', 'PEbs', 'PEbspost', 'PFOFamps', 'PFWKf20', 'PFWKf40', 'PFWKr20', 'PFWKr40', 'PFbs', 'PFbspost', 'PFnorm', 'PSUMbs', 'PTINTall', 'PTINTall_PTbscorr', 'PTINTall_PTdbscorr', 'PTOFamps', 'PTOFamps0', 'PTWKf20', 'PTWKf40', 'PTWKr20', 'PTWKr40', 'PTdbs', 'PTwid', 'T', '_AXIS_LEN', '_AXIS_ORDERS', '_AXIS_REVERSED', '_AXIS_TO_AXIS_NUMBER', '_HANDLED_TYPES', '__abs__', '__add__', '__and__', '__annotations__', '__array__', '__array_priority__', '__array_ufunc__', '__array_wrap__', '__bool__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__divmod__', '_

KeyError: 'PTOFdelay'